# Mock critic
Let's make a critic which always outputs the true discounted reward (assuming that the actor never trades). Then we can train an actor and see if it learns - it should get positive TD errors when it makes good trades which improve the overall balance.

If the agent doesn't learn, it might be due to the critic choosing to discourage actions almost all the time (most actions are bad actions). So if this doesn't work, it doesn't mean the agent is necessarily bad.

In [1]:
import numpy as np
import torch
from tqdm.notebook import trange
from tradezoo.agent import Action, Actor, Agent, Observation
from tradezoo.game import Game, Client, SineWave, Trader
from tradezoo.market import Account, Market
from tradezoo.plots import balance_plot, reward_plot, trades_plot, training_plot
from tradezoo.trainer import Experience, Trainer

In [2]:
discount_factor = 0.99

In [3]:
class MockCritic(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        values = observations.exp() - Observation.epsilon
        result = torch.log(1 + values[:, :2].sum(axis=1)) / (1 - discount_factor)
        result.requires_grad = True
        return result

mock_critic = MockCritic()

In [4]:
class MockOptimizer:
    def step(self):
        pass

    def zero_grad(self):
        pass

mock_optimizer = MockOptimizer()

In [5]:
actor = Actor()
agent = Agent(
    actor=actor,
    actor_optimizer=torch.optim.Adam(actor.parameters(), lr=1e-3),
    critic=mock_critic,
    critic_optimizer=mock_optimizer,
    discount_factor=discount_factor,
    uncertainty=1e-3,
)

In [6]:
trader_account = Account(cash_balance=64, asset_balance=64)
client_account = Account(cash_balance=float("inf"), asset_balance=float("inf"))
price_process = SineWave(period=16).exp()

Let's mock the trader's utility function to ignore the current asset price, in line with the critic's output. This way, the TD error won't be as confusing to the actor - it will always be positive if the action got us closer to the final goal, and negative if it got us further away.

In [7]:
class MockTrader(Trader):
    def utility(self, observation):
        return np.log(1 + observation.cash_balance + observation.asset_balance)

trader = MockTrader(
    agent=agent,
    account=trader_account,
    client=Client(
        account=client_account,
        for_account=trader_account,
        ask_process=price_process * 1.1,
        bid_process=price_process / 1.1,
    ),
)

In [8]:
def mock_experience() -> Experience:
    step = np.random.randint(0, 1024)
    old_observation = Observation(
        cash_balance=np.random.uniform(0, trader_account.cash_balance * 2),
        asset_balance=np.random.uniform(0, trader_account.asset_balance * 2),
        best_ask=trader.client.ask_process.value(step),
        best_bid=trader.client.bid_process.value(step),
    )
    (action,) = agent.decide(old_observation.batch).sample()

    new_cash_balance = old_observation.cash_balance
    new_asset_balance = old_observation.asset_balance
    target_asset_balance = action.constrained_asset_allocation * (
        old_observation.cash_balance + old_observation.asset_balance
    )
    target_trade_volume = abs(old_observation.asset_balance - target_asset_balance)
    if target_asset_balance < old_observation.asset_balance:
        sold_assets = min(new_asset_balance, target_trade_volume)
        new_cash_balance += old_observation.best_bid / 1.1 * sold_assets
        new_asset_balance -= sold_assets
    else:
        bid_price = old_observation.best_ask * 1.1
        bought_assets = min(new_cash_balance / bid_price, target_trade_volume)
        new_cash_balance -= bid_price * bought_assets
        new_asset_balance += bought_assets
    
    new_observation = Observation(
        cash_balance=new_cash_balance,
        asset_balance=new_asset_balance,
        best_ask=trader.client.ask_process.value(step + 1),
        best_bid=trader.client.bid_process.value(step + 1),
    )
    return Experience(
        old_observation=old_observation,
        action=action,
        reward=trader.utility(new_observation),
        new_observation=new_observation,
    )


train_results = [
    Trainer.train_(agent, experiences=[mock_experience() for _ in range(32)])
    for _ in trange(8192)
]


  0%|          | 0/8192 [00:00<?, ?it/s]

In [9]:
training_plot(train_results)

In [10]:
game = Game.new(
    market=Market.from_accounts([trader_account, client_account]),
    traders=[trader],
)
turn_results = [game.turn_() for _ in trange(1024)]

  0%|          | 0/1024 [00:00<?, ?it/s]

In [11]:
trades_plot(turn_results)

In [12]:
reward_plot(turn_results)

In [13]:
balance_plot(turn_results)